In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')

In [2]:
%matplotlib nbagg
import jungfrau_utils as ju
import numpy as np
import json
import os
import time
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

import h5py

from alvra_tools.load_data import *
from alvra_tools.channels import *
from built_functions import *

[INFO][corrections] Numba not available, reverting to Numpy


In [3]:
def get_xes_pumped(filename, DIR, DIRBS, roi, channel, ynstamp):
    numstds = 2.5
    minIzero = 0.01
    lin_filter = 0.2
    image_threshold = 2
    hot_pixel = 6

    print(DIR + filename + ".JF02T09V02crop.h5")
    print()
    print(DIRBS + filename + ".BSREAD.h5")
    
    images_off, images_on, pulse_ids_off, pulse_ids_on = \
        load_JF_cropped_data_pump(DIR + filename + ".JF02T09V02crop.h5", roi, 50, 50/3, nshots=None)
    (DataFluo_pump, DataFluo_unpump, IzeroFEL_pump, IzeroFEL_unpump, JFIDs_pump, JFIDs_unpump, Energy_pump, Energy_unpump, _, _) = \
        load_PumpProbe_events_withTwoVariables(DIRBS + filename + ".BSREAD.h5", channel_BS_pulse_ids, channel)
    (_, _, _, _, _, _, Laser_pump, Laser_unpump, _, _) = \
        load_PumpProbe_events_withTwoVariables(DIRBS + filename + ".BSREAD.h5", channel_PIPS_fluo, channel)
    
    Energy_pump = [x for x in Energy_pump if (np.abs(x) > 0)]
    Energy_eV_pump = np.mean(Energy_pump)
    
    Energy_unpump = [x for x in Energy_unpump if (np.abs(x) > 0)]
    Energy_eV_unpump = np.mean(Energy_unpump)
    
    idcheck = [a==b and c==d for a,b,c,d in zip(pulse_ids_on, JFIDs_pump, pulse_ids_off, JFIDs_unpump)]
    IDCheck = sum([int(x) for x in idcheck])
    
    if IDCheck:
        print("passed ID check")
    else:
        print("failed ID check")   
    
    IzeroMedian = np.median(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))
    IzeroSTD = np.std(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))
    
    print('numon')
    print(IzeroFEL_pump.shape[0])
    
    print('numoff')
    print(IzeroFEL_unpump.shape[0])
    
    print('number of frames')
    print(images_on.shape[0])
    print(images_off.shape[0])

    images_good_on = images_on
    images_good_off = images_off

    images_thr_on = images_good_on.copy()
    images_thr_on[images_good_on < image_threshold] = 0
    images_thr_on[images_good_on > hot_pixel] = 0
    images_thr_on[np.isnan(images_thr_on)] = 0
    
    images_thr_off = images_good_off.copy()
    images_thr_off[images_good_off < image_threshold] = 0
    images_thr_off[images_good_off > hot_pixel] = 0
    images_thr_off[np.isnan(images_thr_off)] = 0
    
    print('num nan')
    print(sum(sum(sum(np.isnan(images_thr_on)))))
    
    print('Izero')
    print(np.mean(IzeroFEL_unpump))
    print(np.mean(IzeroFEL_pump))
    print(np.max(IzeroFEL_unpump))
    print(np.min(IzeroFEL_unpump))
    
    IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, condFinalPump, condFinalUnPump = \
        filterData(IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, numstds, minIzero, lin_filter)
    

    
    print(images_thr_on.shape)
    print(images_thr_on.shape)
    
    images_thr_on = images_thr_on[condFinalPump]
    images_thr_off = images_thr_off[condFinalUnPump]

    print('number of surviving frames')
    print(images_thr_on.shape[0])
    print(images_thr_off.shape[0])
    
    XES_on = images_thr_on.sum(axis=0)/images_thr_on.shape[0]
    XES_off = images_thr_off.sum(axis=0)/images_thr_off.shape[0]
    
    XES_on[np.isnan(XES_on)] = 0
    XES_off[np.isnan(XES_off)] = 0
    
    print('num nan')
    print(sum(sum(np.isnan(XES_on))))
    
    stamp = make_bar_stamp(XES_on.shape[1],XES_on.shape[0])
    
    print(XES_on.shape)
    print(stamp.shape)
    if ynstamp:

        XES_on = XES_on * stamp
        XES_off = XES_off * stamp
        
    FloFEL_pump_total = np.mean(DataFluo_pump)/np.mean(IzeroFEL_pump)
    FloFEL_unpump_total = np.mean(DataFluo_unpump)/np.mean(IzeroFEL_unpump)
    
    return XES_on, XES_off, Energy_eV_pump, Energy_eV_unpump, FloFEL_pump_total, FloFEL_unpump_total


In [4]:
def make_bar_stamp(dimension_x, dimension_y):

    X,Y = np.meshgrid(np.linspace(0,dimension_x-1,dimension_x),np.linspace(0,dimension_y-1,dimension_y))

    Z = np.zeros([dimension_y,dimension_x])+1

    condition = np.logical_and(Y<260, Y>253)
    Z[condition] = 0
    
    Z[Y>320] = 0
    Z[Y < 200] = 0

    condition = np.logical_not(np.logical_and(X<200, X>130))
    Z[condition] = 0

    #plt.figure()
    #plt.imshow(Z)
    
    #plt.colorbar()
    
    return Z

Run for RIXS

In [5]:
scannum_01 = [6,7]

scannum = scannum_01

for jj in range(len(scannum)):
    
    filename_base = "RuDimerACN_monoscan_0p6ps_0" + '%02d' % scannum[jj] + "_step00"
    scan_name = "RuDimerACN_monoscan_0p6ps_0" + '%02d' % scannum[jj]
    DIR = "/das/work/p17/p17983/cropped_data/scan_data/" + scan_name + "/"
    DIRBS = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"
    numfiles = 54+1
    #numfiles = 3

    eV_on_01 = np.empty(numfiles)
    eV_off_01 = np.empty(numfiles)
    
    fluo_Pump = np.empty(numfiles)
    fluo_Unpump = np.empty(numfiles)
    

    for ii in range(numfiles):

        XES_on, XES_off, Energy_eV_pump, Energy_eV_unpump, fluo_pump, fluo_unpump = \
            get_xes_pumped(filename_base + '%02d' % (ii+0), DIR, DIRBS, 2, channel_energy, True)

        if False:

            plt.figure()
            plt.subplot(1,2,1)
            plt.imshow(XES_on, vmax = 0.001)
            plt.colorbar()
            plt.subplot(1,2,2)
            plt.imshow(XES_off, vmax = 0.001)
            plt.colorbar()
            plt.tight_layout()
        
        
        if ii == 0 & jj == 0:
            rixs_on_01 = XES_on.sum(axis=0)
            rixs_off_01 = XES_off.sum(axis=0)
        else:
            rixs_on_01 = np.vstack((rixs_on_01,XES_on.sum(axis=0)))
            rixs_off_01 = np.vstack((rixs_off_01,XES_off.sum(axis=0)))
            
        eV_on_01[ii] = Energy_eV_pump
        eV_off_01[ii] = Energy_eV_unpump
        fluo_Pump[ii] = fluo_pump
        fluo_Unpump[ii] = fluo_unpump
            
        
    if jj == 0:
        
        RIXS_on_01 = rixs_on_01
        RIXS_off_01 = rixs_off_01
        Fluo_pump_01 = fluo_Pump
        Fluo_unpump_01 = fluo_Unpump
        
    else:
        
        RIXS_on_01 = RIXS_on_01 + rixs_on_01
        RIXS_off_01 = RIXS_off_01 + rixs_off_01
        Fluo_pump_01 = Fluo_pump_01 + fluo_Pump
        Fluo_unpump_01 = Fluo_unpump_01 + fluo_Unpump



/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0000.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0000.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.0453105069271795
0.04551742642397421
0.1212494745850563
0.015268517658114433
(667, 512, 300)
(667, 512, 300)
number of surviving frames
652
323
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0001.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0001.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04

/sf/alvra/anaconda/dev/lib/python3.6/site-packages/ipykernel/__main__.py:83: RuntimeWarning: invalid value encountered in true_divide
/sf/alvra/anaconda/dev/lib/python3.6/site-packages/ipykernel/__main__.py:84: RuntimeWarning: invalid value encountered in true_divide
/sf/alvra/anaconda/dev/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/sf/alvra/anaconda/dev/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04256296738445222
0.04244765633440715
0.11480414867401123
0.008804102428257465
(667, 512, 300)
(667, 512, 300)
number of surviving frames
646
318
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0014.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0014.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.04324667214577694
0.041718961243024245
0.0969846174120903
0.00878725666552782
(666, 512, 300)
(666, 512, 300)
number of surviving frames
648
321
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerA

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.04258513821251997
0.04291597808155898
0.1298571228981018
0.0005047518643550575
(666, 512, 300)
(666, 512, 300)
number of surviving frames
647
323
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0028.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0028.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.043154848322786904
0.04288300064795914
0.11308262497186661
0.00546787865459919
(667, 512, 300)
(667, 512, 300)
number of surviving frames
646
322
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimer

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.03956763965130837
0.041044867362944354
0.10108054429292679
0.009434295818209648
(667, 512, 300)
(667, 512, 300)
number of surviving frames
647
324
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0042.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDimerACN_monoscan_0p6ps_006_step0042.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.043358009029737486
0.04213821747292717
0.12290129065513611
0.014592912048101425
(666, 512, 300)
(666, 512, 300)
number of surviving frames
643
323
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_006/RuDim

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.042243705058397654
0.043816530420773034
0.09438301622867584
0.009304875507950783
(667, 512, 300)
(667, 512, 300)
number of surviving frames
646
322
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimerACN_monoscan_0p6ps_007_step0001.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimerACN_monoscan_0p6ps_007_step0001.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04279508127330302
0.044915460315400335
0.11777578294277191
0.012758544646203518
(667, 512, 300)
(667, 512, 300)
number of surviving frames
646
324
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDi

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.04378594747816999
0.042367805679281195
0.11717740446329117
0.011450519785284996
(666, 512, 300)
(666, 512, 300)
number of surviving frames
650
325
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimerACN_monoscan_0p6ps_007_step0015.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimerACN_monoscan_0p6ps_007_step0015.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
666
numoff
334
number of frames
666
334
num nan
0
Izero
0.0440413084182286
0.04309707934277373
0.11607447266578674
0.006133544258773327
(666, 512, 300)
(666, 512, 300)
number of surviving frames
648
324
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimer

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04237171586647674
0.041816578516136224
0.14841248095035553
0.013164769858121872
(667, 512, 300)
(667, 512, 300)
number of surviving frames
652
323
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimerACN_monoscan_0p6ps_007_step0029.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimerACN_monoscan_0p6ps_007_step0029.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.03910656199810115
0.04028091444375588
0.1125129908323288
0.010548979043960571
(667, 512, 300)
(667, 512, 300)
number of surviving frames
648
321
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimer

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04038831196095572
0.04108483531139512
0.12345924973487854
0.00013213402417022735
(667, 512, 300)
(667, 512, 300)
number of surviving frames
651
320
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimerACN_monoscan_0p6ps_007_step0043.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDimerACN_monoscan_0p6ps_007_step0043.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.03989927369161501
0.04035958885794681
0.11390885710716248
0.011706012301146984
(667, 512, 300)
(667, 512, 300)
number of surviving frames
650
323
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_0p6ps_007/RuDim

In [6]:
print(sum(sum(np.isnan(RIXS_off_01))))

0


In [ ]:
scannum_02 = [5]

scannum = scannum_02

for jj in range(len(scannum)):
    
    filename_base = "RuDimerACN_monoscan_10ps_0" + '%02d' % scannum[jj] + "_step00"
    scan_name = "RuDimerACN_monoscan_10ps_0" + '%02d' % scannum[jj]
    DIR = "/das/work/p17/p17983/cropped_data/scan_data/" + scan_name + "/"
    DIRBS = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"
    numfiles = 54+1
    #numfiles = 3

    eV_on_02 = np.empty(numfiles)
    eV_off_02 = np.empty(numfiles)
    
    fluo_Pump = np.empty(numfiles)
    fluo_Unpump = np.empty(numfiles)
    

    for ii in range(numfiles):

        XES_on, XES_off, Energy_eV_pump, Energy_eV_unpump, fluo_pump, fluo_unpump = \
            get_xes_pumped(filename_base + '%02d' % (ii+0), DIR, DIRBS, 2, channel_energy, True)

        if False:

            plt.figure()
            plt.subplot(1,2,1)
            plt.imshow(XES_on, vmax = 0.001)
            plt.colorbar()
            plt.subplot(1,2,2)
            plt.imshow(XES_off, vmax = 0.001)
            plt.colorbar()
            plt.tight_layout()
        
        
        if ii == 0 & jj == 0:
            rixs_on_02 = XES_on.sum(axis=0)
            rixs_off_02 = XES_off.sum(axis=0)
        else:
            rixs_on_02 = np.vstack((rixs_on_02,XES_on.sum(axis=0)))
            rixs_off_02 = np.vstack((rixs_off_02,XES_off.sum(axis=0)))
            
        eV_on_02[ii] = Energy_eV_pump
        eV_off_02[ii] = Energy_eV_unpump
        fluo_Pump[ii] = fluo_pump
        fluo_Unpump[ii] = fluo_unpump
            
        
    if jj == 0:
        
        RIXS_on_02 = rixs_on_02
        RIXS_off_02 = rixs_off_02
        Fluo_pump_02 = fluo_Pump
        Fluo_unpump_02 = fluo_Unpump
        
    else:
        
        RIXS_on_02 = RIXS_on_02 + rixs_on_02
        RIXS_off_02 = RIXS_off_02 + rixs_off_02
        Fluo_pump_02 = Fluo_pump_02 + fluo_Pump
        Fluo_unpump_02 = Fluo_unpump_02 + fluo_Unpump

/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0000.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0000.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04350770250320793
0.044284792911579676
0.10642220079898834
0.012368666008114815
(667, 512, 300)
(667, 512, 300)
number of surviving frames
645
324
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0001.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0001.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.0445269

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04221367844109152
0.043603017035385834
0.10860086232423782
0.008448210544884205
(667, 512, 300)
(667, 512, 300)
number of surviving frames
642
322
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0015.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0015.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04538416308448748
0.04395134396002121
0.13053317368030548
0.013096618466079235
(667, 512, 300)
(667, 512, 300)
number of surviving frames
650
317
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04273146049989326
0.04261483220352807
0.10036314278841019
0.011644076555967331
(667, 512, 300)
(667, 512, 300)
number of surviving frames
646
320
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0029.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0029.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.043117345391853794
0.042296987500270715
0.10971636325120926
0.013133049942553043
(667, 512, 300)
(667, 512, 300)
number of surviving frames
646
321
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.04199321616955616
0.04038160390638072
0.13085320591926575
0.009535681456327438
(667, 512, 300)
(667, 512, 300)
number of surviving frames
654
319
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0043.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_monoscan_10ps_005_step0043.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
667
numoff
333
number of frames
667
333
num nan
0
Izero
0.041273167946966026
0.04149554290678185
0.1116122230887413
0.013812558725476265
(667, 512, 300)
(667, 512, 300)
number of surviving frames
642
323
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerACN_monoscan_10ps_005/RuDimerACN_m

In [ ]:
X,Y = np.meshgrid(np.linspace(0,RIXS_on_01.shape[1],RIXS_on_01.shape[1]+1),eV_on_01)
plt.figure()
plt.subplot(2,1,1)
plt.pcolor(X,Y,RIXS_on_01, vmax = 1)
plt.colorbar()
plt.xlabel('JF pixel')
plt.ylabel('Mono Energy (eV)')
plt.title('scannum ' + str(scannum_01) + ' on')

X,Y = np.meshgrid(np.linspace(0,RIXS_off_01.shape[1],RIXS_off_01.shape[1]+1),eV_off_01)
plt.subplot(2,1,2)
plt.pcolor(X,Y,RIXS_off_01, vmax = 1)
plt.colorbar()
plt.xlabel('JF pixel')
plt.ylabel('Mono Energy (eV)')
plt.title('scannum ' + str(scannum_01) + ' off')

plt.tight_layout()


X,Y = np.meshgrid(np.linspace(0,RIXS_on_02.shape[1],RIXS_on_02.shape[1]+1),eV_on_02)
plt.figure()
plt.subplot(2,1,1)
plt.pcolor(X,Y,RIXS_on_02, vmax = 1)
plt.colorbar()
plt.xlabel('JF pixel')
plt.ylabel('Mono Energy (eV)')
plt.title('scannum ' + str(scannum_02) + ' on')

X,Y = np.meshgrid(np.linspace(0,RIXS_off_02.shape[1],RIXS_off_02.shape[1]+1),eV_off_02)
plt.subplot(2,1,2)
plt.pcolor(X,Y,RIXS_off_02, vmax = 1)
plt.colorbar()
plt.xlabel('JF pixel')
plt.ylabel('Mono Energy (eV)')
plt.title('scannum ' + str(scannum_02) + ' off')

plt.tight_layout()

In [ ]:
# # plt.tight_layout()
# #X,Y = np.meshgrid(np.linspace(0,RIXS_on_02.shape[1],RIXS_on_02.shape[1]+1),eV_on_02)
# plt.figure()
# #plt.subplot(2,1,1)
# plt.pcolor(RIXS_on_02, vmax = 1)
# plt.colorbar()
# plt.xlabel('JF pixel')
# plt.ylabel('Mono Energy (eV)')
# plt.title('scannum ' + str(scannum_02) + ' on')

# HERFD_on_02 = np.empty(0)

# n1 = RIXS_on_02.shape[0]
# n2 = RIXS_on_02.shape[1]

# print(n1)
# print(n2)

# for jj in range(0, n1 - 1):
#     j1 = int(4/3*jj + 128.67) - 2
#     j2 = int(4/3*jj + 128.67) + 2
#     test = RIXS_on_02[jj, j1:j2].mean(axis=0)
#     print(test)
#     HERFD_on_02 = np.append(HERFD_on_02, test.mean(axis=1))
    
# plt.figure()
# plt.plot(eV_on_02, HERFD_on_02)

# print(RIXS_on_02.shape)

In [ ]:
plt.figure(figsize=(8,12))
plt.subplot(4,1,1)
plt.plot(eV_on_01, RIXS_on_01.sum(axis=1), label = "on")
plt.plot(eV_off_01, RIXS_off_01.sum(axis=1), label = 'off')
plt.xlabel('Mono Energy')
plt.title('projected XAS scannum ' + str(scannum_01))
plt.legend()

plt.subplot(4,1,2)
plt.plot(eV_off_01, RIXS_on_01.sum(axis=1)-RIXS_off_01.sum(axis=1))
plt.xlabel('Mono Energy')
plt.title('projected XAS scannum ' + str(scannum_01) + ' on - off')

plt.subplot(4,1,3)
plt.plot(eV_on_01, Fluo_pump_01, label = "on")
plt.plot(eV_off_01, Fluo_unpump_01, label = 'off')
plt.xlabel('Mono Energy')
plt.title('XAS TFY ' + str(scannum_01))
plt.legend()

plt.subplot(4,1,4)
plt.plot(eV_off_01, Fluo_pump_01-Fluo_unpump_01)
plt.xlabel('Mono Energy')
plt.title('XAS TFY ' + str(scannum_01) + ' on - off')


plt.tight_layout()


plt.figure(figsize=(8,12))
plt.subplot(4,1,1)
plt.plot(eV_on_02, RIXS_on_02.sum(axis=1), label = "on")
plt.plot(eV_off_02, RIXS_off_02.sum(axis=1), label = 'off')
plt.xlabel('Mono Energy')
plt.title('projected XAS scannum ' + str(scannum_02))
plt.legend()

plt.subplot(4,1,2)
plt.plot(eV_off_02, RIXS_on_02.sum(axis=1)-RIXS_off_02.sum(axis=1))
plt.xlabel('Mono Energy')
plt.title('projected XAS scannum ' + str(scannum_02) + ' on - off')

plt.subplot(4,1,3)
plt.plot(eV_on_02, Fluo_pump_02, label = "on")
plt.plot(eV_off_02, Fluo_unpump_02, label = 'off')
plt.xlabel('Mono Energy')
plt.title('XAS TFY ' + str(scannum_02))
plt.legend()

plt.subplot(4,1,4)
plt.plot(eV_off_02, Fluo_pump_02-Fluo_unpump_02)
plt.xlabel('Mono Energy')
plt.title('XAS TFY ' + str(scannum_02) + ' on - off')

plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,6))
plt.subplot(2,1,1)
plt.plot(RIXS_on_01.sum(axis=0), label = "on")
plt.plot(RIXS_off_01.sum(axis=0), label = "off")
plt.xlabel('JF pixels')
plt.title('projected XES scannum ' + str(scannum_01))
plt.xlim([130,200])

plt.subplot(2,1,2)
plt.plot(RIXS_on_01.sum(axis=0) - RIXS_off_01.sum(axis=0))
plt.xlabel('JF pixels')
plt.title('projected XES TFY ' + str(scannum_01) + ' on - off')
plt.xlim([130,200])

plt.tight_layout()

plt.figure(figsize=(8,6))
plt.subplot(2,1,1)
plt.plot(RIXS_on_02.sum(axis=0), label = "on")
plt.plot(RIXS_off_02.sum(axis=0), label = "off")
plt.xlabel('JF pixels')
plt.title('projected XES scannum ' + str(scannum_02))
plt.xlim([130,200])

plt.subplot(2,1,2)
plt.plot(RIXS_on_02.sum(axis=0) - RIXS_off_02.sum(axis=0))
plt.xlabel('JF pixels')
plt.title('projected XES TFY ' + str(scannum_02) + ' on - off')
plt.xlim([130,200])

plt.tight_layout()

In [ ]:
pixel_range_max = 170
pixel_range_min = 160


plt.figure(figsize=(8,12))
plt.subplot(4,1,1)
plt.plot(eV_on_01, RIXS_on_01[:,pixel_range_min:pixel_range_max].sum(axis=1), label = "on")
plt.plot(eV_off_01, RIXS_off_01[:,pixel_range_min:pixel_range_max].sum(axis=1), label = 'off')
plt.xlabel('Mono Energy')
plt.title('HERFD scannum ' + str(scannum_01))
plt.legend()

plt.subplot(4,1,2)
plt.plot(eV_off_01, RIXS_on_01[:,pixel_range_min:pixel_range_max].sum(axis=1)-RIXS_off_01[:,pixel_range_min:pixel_range_max].sum(axis=1))
plt.xlabel('Mono Energy')
plt.title('HERFD scannum ' + str(scannum_01) + ' on - off')

plt.subplot(4,1,3)
plt.plot(eV_on_01, Fluo_pump_01, label = "on")
plt.plot(eV_off_01, Fluo_unpump_01, label = 'off')
plt.xlabel('Mono Energy')
plt.title('XAS TFY ' + str(scannum_01))
plt.legend()

plt.subplot(4,1,4)
plt.plot(eV_off_01, Fluo_pump_01-Fluo_unpump_01)
plt.xlabel('Mono Energy')
plt.title('XAS TFY ' + str(scannum_01) + ' on - off')


plt.tight_layout()


plt.figure(figsize=(8,12))
plt.subplot(4,1,1)
plt.plot(eV_on_02, RIXS_on_02[:,pixel_range_min:pixel_range_max].sum(axis=1), label = "on")
plt.plot(eV_off_02, RIXS_off_02[:,pixel_range_min:pixel_range_max].sum(axis=1), label = 'off')
plt.xlabel('Mono Energy')
plt.title('constant emission energy scannum ' + str(scannum_02))
plt.legend()

plt.subplot(4,1,2)
plt.plot(eV_off_02, RIXS_on_02[:,pixel_range_min:pixel_range_max].sum(axis=1)-RIXS_off_02[:,pixel_range_min:pixel_range_max].sum(axis=1))
plt.xlabel('Mono Energy')
plt.title('constant emission energy scannum ' + str(scannum_02) + ' on - off')

plt.subplot(4,1,3)
plt.plot(eV_on_02, Fluo_pump_02, label = "on")
plt.plot(eV_off_02, Fluo_unpump_02, label = 'off')
plt.xlabel('Mono Energy')
plt.title('XAS TFY ' + str(scannum_02))
plt.legend()

plt.subplot(4,1,4)
plt.plot(eV_off_02, Fluo_pump_02-Fluo_unpump_02)
plt.xlabel('Mono Energy')
plt.title('XAS TFY ' + str(scannum_02) + ' on - off')

plt.tight_layout()

In [ ]:
mono_range_max = 2842.5
mono_range_min = 2841.5


index_min = min(min(np.where(eV_on_01 < mono_range_max)))
index_max = max(max(np.where(eV_on_01 > mono_range_min)))

print('mono_max')
print(eV_on_01[index_min])
print('mono_min')
print(eV_on_01[index_max])

plt.figure(figsize=(8,6))
plt.subplot(2,1,1)
plt.plot(RIXS_on_01[index_min:index_max].sum(axis=0), label = "on")
plt.plot(RIXS_off_01[index_min:index_max].sum(axis=0), label = "off")
plt.xlabel('JF pixels')
plt.title('mono XES scannum ' + str(scannum_01))
plt.xlim([130,200])
plt.legend()

plt.subplot(2,1,2)
plt.plot(RIXS_on_01[index_min:index_max].sum(axis=0) - RIXS_off_01[index_min:index_max].sum(axis=0))
plt.xlabel('JF pixels')
plt.title('mono XES scannum ' + str(scannum_01) + 'on - off')
plt.xlim([130,200])

plt.tight_layout()

index_min = min(min(np.where(eV_on_02 < mono_range_max)))
index_max = max(max(np.where(eV_on_02 > mono_range_min)))

print('mono_max')
print(eV_on_02[index_min])
print('mono_min')
print(eV_on_02[index_max])

plt.figure(figsize=(8,6))
plt.subplot(2,1,1)
plt.plot(RIXS_on_02[index_min:index_max].sum(axis=0), label = "on")
plt.plot(RIXS_off_02[index_min:index_max].sum(axis=0), label = "off")
plt.xlabel('JF pixels')
plt.title('mono XES scannum ' + str(scannum_02))
plt.xlim([130,200])
plt.legend()

plt.subplot(2,1,2)
plt.plot(RIXS_on_02[index_min:index_max].sum(axis=0) - RIXS_off_02[index_min:index_max].sum(axis=0))
plt.xlabel('JF pixels')
plt.title('mono XES scannum ' + str(scannum_02) + 'on - off')
plt.xlim([130,200])

plt.tight_layout()
print(index_min)
print(index_max)

In [ ]:
if False:
    np.save('/das/work/p17/p17983/Reduced_data/ACN_RIXS_0p6ps_on_scan_06_07.npy', RIXS_on_01)
    np.save('/das/work/p17/p17983/Reduced_data/ACN_RIXS_0p6ps_off_scan_06_07.npy', RIXS_off_01)
    np.save('/das/work/p17/p17983/Reduced_data/ACN_RIXS_10ps_on_scan_05.npy', RIXS_on_02)
    np.save('/das/work/p17/p17983/Reduced_data/ACN_RIXS_10ps_off_scan_05.npy', RIXS_off_02)